In [7]:
def getDegreeMatrix(graph):
    degreeMatrix = np.zeros((len(graph.nodes()),len(graph.nodes())))
    for i in range(len(graph.nodes())):
        degreeMatrix[i][i] = graph.degree(graph.nodes()[i])
    return degreeMatrix

def getEdgeToEdgeProbMatrix(G, alpha,lazy = False):
    D = getDegreeMatrix(G)
    W = getWeightMatrix(G)
    A = nx.adjacency_matrix(G).toarray()
    P = np.dot(np.linalg.inv(D), A)
    B, E = getIncidenceMatriсes(G)
    H = np.dot(np.dot(np.linalg.inv(D),B),W)
    minusalphaE = np.dot(1-alpha,E)
    minusalphaEH = np.dot(minusalphaE, H)
    EB = np.dot(E,B)
    alphaEB = np.dot(alpha, EB)
    Bt = np.transpose(B)
    BtP = np.dot(Bt,P)
    Et = np.transpose(E)
    BtPEt = np.dot(BtP,Et)
    alphaEBxBtPEt = np.multiply(alphaEB, BtPEt)
    M=np.add(minusalphaEH, alphaEBxBtPEt)
    if (lazy == True):
        M = np.divide(np.add(M, np.identity(len(M))), 2)
    return rowNormalize(M)

def getNodeToNodeProbMatrix(G):
    D = getDegreeMatrix(G)
    A = nx.adjacency_matrix(G).toarray()
    P = np.dot(np.add(np.identity(len(G.nodes())), A), np.linalg.inv(np.add(np.identity(len(G.nodes())), D)  ))
    return P

import copy
def explore(probability_matrix, i, T=100, r = 2, e=.00001, epsilon=.00001):
    x = np.zeros(len(probability_matrix))
    x[i] = 1
    x_prev = copy.deepcopy(x)
    for t in range(T):
        x = np.dot(probability_matrix, x)
        x = (x * (x > e))**r
        x = x/sum(x)
        if max(x - x_prev) < epsilon:
            return x_prev
        x_prev = copy.deepcopy(x)
    return x
def merge(G, treshold):
    D = dict()
    P = getNodeToNodeProbMatrix(G)
    for index in range(len(G.nodes())):
        F = set()
        S = set()
        x = explore(P, index)
        m = np.argmax(x)
        S.add(index)
        for j in range(len(x)):
            if x[j] > treshold * x[m]:
                F.add(j)
        if m not in D.keys():
            D[m] = [S, F]
        else:
            D[m][0].update(S)
            D[m][1].update(F)
    
    passed = dict()
    dkeys1 = list(D.keys())
    for m1 in dkeys1:
        if m1 in D.keys(): #if m1 still in keys
            dkeys2 = list(D.keys())
            for m2 in dkeys2:
                if m2 in D.keys() and m2 != m1: #if m1 still in keys
                    if m1 in passed.keys():
                        if m2 in passed[m1]:
                            break;
                    elif m2 in passed.keys():
                        if m1 in passed[m2]:
                            break;
                    if m1 not in passed.keys():
                        passed[m1] = set()
                    passed[m1].add(m2)
                    if len(D[m1][1] & D[m2][1]) > min(len(D[m1][1]),len(D[m2][1]))/2:
                        D[m1][0].update(D[m2][0])
                        D[m1][1].update(D[m2][1])
                        del D[m2]
    return D
def get_name(person):
    name = 'unknown'
    with urllib.request.urlopen("https://api.vk.com/method/users.get?user_id="+str(person)+"&v=5.52") as url:
            received = json.loads(url.read().decode())
            if 'error' not in received:
                name = str(received['response'][0]['first_name'])+' '+str(received['response'][0]['last_name'])
    return name

In [10]:
import networkx as nx
import urllib.request, json
import numpy as np
import pandas as pd
user_id = 3208690


G=nx.Graph()
with urllib.request.urlopen("https://api.vk.com/method/friends.get?user_id="+str(user_id)+"&v=5.52") as url:
    G.add_nodes_from(json.loads(url.read().decode())['response']['items'])

for node in G.nodes():
    with urllib.request.urlopen("https://api.vk.com/method/friends.get?user_id="+str(node)+"&v=5.52") as url:
        received = json.loads(url.read().decode())
        if 'error' not in received:
            for friend in received['response']['items']:
                if friend in G.nodes():
                    G.add_edge(node, friend)
                    
print("Nodes: " + str(len(G.nodes())))
print("Edges: " + str(len(G.edges())))

graphs = list(nx.connected_component_subgraphs(G))
for graph in graphs:
    print("Connected graph " + str(graphs.index(graph)))
    communities = merge(graph, 0.5)
    keys = list(communities.keys())
    for key_ind in range(len(keys)):
        print("Community " + str(key_ind + 1) + ":")
        for person_index in communities[keys[key_ind]][0]:
            print("      " + str(graph.nodes()[person_index])+ ": " + get_name(graph.nodes()[person_index]))
        print('\n')
    print("\n\n")

Nodes: 41
Edges: 57
Connected graph 0
Community 1:
      2509793: Антон Мартынов
      306745: Таня Евсюкова


Community 2:
      257506364: Михаил Димидов


Community 3:
      464017: Султан Сулейманов
      93388: Владислав Цыплухин
      15857579: Anvar Valiev
      59628: Максим Бабичев
      260079373: Степан Савельев
      1300017: Никита Лихачёв
      18178869: Лера Аксенфельд
      551225: Антон Меркуров
      151082068: Алексей Свирин
      5488052: Владимир Климов
      72446: Екатерина Патюлина


Community 4:
      6204621: Сергей Чепур
      17085716: Майя Вольф


Community 5:
      24613094: Артем Горбачев
      96509305: Артур Демьянченко
      1683850: Константин Абатуров





Connected graph 1
Community 1:
      131888406: Klm Russia





Connected graph 2
Community 1:
      69334: Юлия Саютинская
      115945110: Паша Акимов





Connected graph 3
Community 1:
      5415176: София Ткач
      2925986: Наталья Самойлова


Community 2:
      1776754: Елизавета Голубцова
 